# Import libraries

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import psycopg2

# Retrieve data from Spotify, 3 songs for each genre

In [2]:
# Set your Spotify API credentials
client_id = '6c2ff081fe494968a2e3a618c678fd62'
client_secret = '99da4179a82f4f999058535d8d9b47dc'

In [3]:
# Initialize the Spotipy client
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id, client_secret))

In [4]:
# Define seed genres
seed_genres = ['anime', 'classical', 'country', 'chill', 'edm', 'folk', 'gospel', 'hip-hop', 'jazz', 'k-pop', 'latin', 'metal', 'pop', 'r-n-b', 'rock']

In [5]:
# Initialize an empty list to store track data
all_track_data = []

In [6]:
# Get recommended tracks for each seed genre
for seed_genre in seed_genres:
    recommended_tracks = sp.recommendations(seed_genres=[seed_genre], limit=3, country="SG")
    for track in recommended_tracks["tracks"]:
        track_data = {
            "track_name": track["name"],
            "artist": ", ".join(artist["name"] for artist in track["artists"]),
            "album": track["album"]["name"],
            "genre": seed_genre,
            "preview_url": track["preview_url"]
        }
        all_track_data.append(track_data)

# Convert Spotify data to dataframe and inspect data

In [7]:
# Create a DataFrame from the track data
df = pd.DataFrame(all_track_data)

In [8]:
# take a look at # of rows/columns
print(df.shape)

(45, 5)


In [9]:
# Print the DataFrame
print(df.head())

                             track_name  \
0                              さよならのゆくえ   
1                              §Rainbow   
2                            FLY HIGH!!   
3  Piano Concerto, Op. 13: 3. Impromptu   
4                               Opus 20   

                                              artist  \
0                                     Alisa Takigawa   
1                                              i☆Ris   
2                                  BURNOUT SYNDROMES   
3  Benjamin Britten, Elizabeth Joy Roe, London Sy...   
4                                  Dustin O'Halloran   

                                             album      genre  \
0                                         さよならのゆくえ      anime   
1                                §Rainbow (TYPE-A)      anime   
2                                       FLY HIGH!!      anime   
3      Britten & Barber Piano Concertos; Nocturnes  classical   
4  Breathe In (Original Motion Picture Soundtrack)  classical   

             

In [10]:
# check % of nulls in each column
total_nulls = df.isnull().sum()
percent_missing = total_nulls * 100 / len(df)
print(percent_missing)

track_name      0.000000
artist          0.000000
album           0.000000
genre           0.000000
preview_url    46.666667
dtype: float64


In [11]:
print(df[df.duplicated()])

Empty DataFrame
Columns: [track_name, artist, album, genre, preview_url]
Index: []


# Connect to DB , create table and load data into DB

In [ ]:
# # Connect to PostgreSQL database
# db_params = {
#     "host": "localhost",
#     "dbname": "mmh_1",  # Replace with your desired database name
#     "user": "postgres",  # Replace with your PostgreSQL username
#     "password": "admin",  # Replace with your PostgreSQL password
#     "port": "5432" # Replace with your PostgreSQL port
# }

db_params = {
    "host": "rain.db.elephantsql.com",
    "dbname": "auspovuc",  # Replace with your desired database name
    "user": "auspovuc",  # Replace with your PostgreSQL username
    "password": "bmJdG19Daw9rkEsJ3VnkefRGCBF_oy7F",  # Replace with your PostgreSQL password
}

In [ ]:
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

In [ ]:
# Create a table (if not exists)
cur.execute("""
    CREATE TABLE IF NOT EXISTS recommended_tracks (
        track_name VARCHAR,
        artist VARCHAR,
        album VARCHAR,
        genre VARCHAR,
        preview_url VARCHAR
    )
""")
conn.commit()

In [ ]:
# # Insert data into the table
for _, row in df.iterrows():
    cur.execute("""
        INSERT INTO recommended_tracks (track_name, artist, album, genre, preview_url)
        VALUES (%s, %s, %s, %s, %s)
    """, (row["track_name"], row["artist"], row["album"], row["genre"], row["preview_url"]))

conn.commit()
conn.close()